### Resources

- http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/


### Skip-Gram Model
- Before skip-gram there was a bi-gram model which uses the most adjacent word to train the model. 
- But in this case the word can be any word inside the window. So you can use any of the words inside the window skipping the most adjacent word. Hence skip-gram.
- We’re going to train a simple neural network with a single hidden layer to perform a certain task, the goal is actually just to learn the weights of the hidden layer–we’ll see that these weights are actually the “word vectors” that we’re trying to learn.
- unsupervised feature learning
  - where you train an auto-encoder to compress an input vector in the hidden layer, and decompress it back to the original in the output layer. After training it, you strip off the output layer (the decompression step) and just use the hidden layer--it's a trick for learning good image features without having labeled training data.
- Given a specific word in the middle of a sentence (the input word), look at the words nearby and pick one at random. The network is going to tell us the probability for every word in our vocabulary of being the “nearby word” that we chose.
- When I say "nearby", there is actually a "window size" parameter to the algorithm. A typical window size might be 5, meaning 5 words behind and 5 words ahead (10 in total).
- “The quick brown fox jumps over the lazy dog.” with windows size of 2
![](http://mccormickml.com/assets/word2vec/training_data.png)
- The network is going to learn the statistics from the number of times each pairing shows up

#### Model details

- we first build a vocabulary of words from our training documents, Let’s say we have a vocabulary of 10,000 unique words.
- We’re going to represent an input word like “ants” as a one-hot vector. This vector will have 10,000 components (one for every word in our vocabulary) 
- The output of the network is a single vector (also with 10,000 components) containing, for every word in our vocabulary, the probability that a randomly selected nearby word is that vocabulary word.
- There is no activation function on the hidden layer neurons, but the output neurons use softmax.
- When training this network on word pairs, the input is a one-hot vector representing the input word and the training output is also a one-hot vector representing the output word. But when you evaluate the trained network on an input word, the output vector will actually be a probability distribution 
![](http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png)

##### Hidden Layer

- we’re going to say that we’re learning word vectors with 300 features. So the hidden layer is going to be represented by a weight matrix with 10,000 rows (one for every word in our vocabulary) and 300 columns (one for every hidden neuron).
- 300 features is what Google used in their published model trained on the Google news dataset (you can download it from here). The number of features is a "hyper parameter" that you would just have to tune to your application (that is, try different values and see what yields the best results).
[![](http://mccormickml.com/assets/word2vec/word2vec_weight_matrix_lookup_table.png)](http://mccormickml.com/assets/word2vec/word2vec_weight_matrix_lookup_table.png)
- what if multiply a 1 x 10,000 one-hot vector by a 10,0000 x 200 matrix, it's just operating as a lookup table.
![](http://mccormickml.com/assets/word2vec/matrix_mult_w_one_hot.png)

##### Output layer
- The output layer is a softmax regression classifier
- each output neuron has a weight vector which it multiplies against the word vector from the hidden layer, then it applies the function exp(x) to the result. Finally, in order to get the outputs to sum up to 1
- neural network does not know anything about the offset of the output word relative to the input word. It does not learn a different set of probabilities for the word before the input versus the word after.
- To understand the implication, let's say that in our training corpus, every single occurrence of the word 'York' is preceded by the word 'New'. That is, at least according to the training data, there is a 100% probability that 'New' will be in the vicinity of 'York'. However, if we take the 10 words in the vicinity of 'York' and randomly pick one of them, the probability of it being 'New' is not 100%; you may have picked one of the other words in the vicinity.
![](http://mccormickml.com/assets/word2vec/output_weights_function.png)

##### Intuition
- If two different words have very similar “contexts” (that is, what words are likely to appear around them), then our model needs to output very similar results for these two words. And one way for the network to output similar context predictions for these two words is **if the word vectors are similar**. So, if two words have similar contexts, then our network is motivated to learn similar word vectors for these two words! Ta da!
- for two words to have similar contexts
  - synonyms like “intelligent” and “smart”
  - fwords that are related, like “engine” and “transmission”
  - handle stemming, like the network will likely learn similar word vectors for the words “ant” and “ants” 
- skip-gram neural network contains a huge number of weights
  - with 300 features and a vocab of 10,000 words, that’s 3M weights in the hidden layer and output layer each
  - Training this on a large dataset would be prohibitive
  

#### Train?
- Running gradient descent on a neural network that large is going to be slow.
- And to make matters worse, you need a huge amount of training data in order to tune that many weights and avoid over-fitting.
- millions of weights times billions of training samples means that training this model is going to be a beast.

#### Solutions
- Treating common word pairs or phrases as single “words” in their model.
- Subsampling frequent words to decrease the number of training examples.
- Modifying the optimization objective with a technique they called “Negative Sampling”, which causes each training sample to update only a small percentage of the model’s weights.

It’s worth noting that subsampling frequent words and applying Negative Sampling not only reduced the compute burden of the training process, but also improved the quality of their resulting word vectors as well.


##### Word Pairs and “Phrases”
- a word pair like “Boston Globe” (a newspaper) has a much different meaning than the individual words “Boston” and “Globe”. So it makes sense to treat “Boston Globe”, wherever it occurs in the text, as a single word with its own word vector representation.
- their published model, which was trained on 100 billion words from a Google News dataset.
- The addition of phrases to the model swelled the vocabulary size to 3 million words!
- Phrase detection
  - Each pass of their tool only looks at combinations of 2 words, but you can run it multiple times to get longer phrases. So, the first pass will pick up the phrase “New_York”, and then running it again will pick up “New_York_City” as a combination of “New_York” and “City”.
  - Each pass of their tool only looks at combinations of 2 words, but you can run it multiple times to get longer phrases. So, the first pass will pick up the phrase “New_York”, and then running it again will pick up “New_York_City” as a combination of “New_York” and “City”.
  - The equation is designed to make phrases out of words which occur together often relative to the number of individual occurrences. 
  - an alternate phrase recognition strategy would be to use the titles of all Wikipedia articles as your vocabulary.
  
- Subsampling Frequent Words
  - review 

- Subsampling Frequent Words
  - review training samples (word pairs) generation
![](http://mccormickml.com/assets/word2vec/training_data.png)
  - two “problems” with common words like “the”
    - When looking at word pairs, (“fox”, “the”) doesn’t tell us much about the meaning of “fox”. “the” appears in the context of pretty much every word.
    - We will have many more samples of (“the”, …) than we need to learn a good vector for “the”
  - Word2Vec implements a “subsampling” scheme to address this. 
    - For each word we encounter in our training text, there is a chance that we will effectively delete it from the text. 
    - The probability that we cut the word is related to the word’s frequency.
    - If we have a window size of 10, and we remove a specific instance of “the” from our text:
      - As we train on the remaining words, “the” will not appear in any of their context windows.
      - We’ll have 10 fewer training samples where “the” is the input word.

##### Sampling rate

- $w_i$  is the word, $z(w_)$ is the fraction of the total words in the corpus that are that word.
- $P(w_i)$ is the probability of keeping the word:

![img_subsampling](files/word2vec_subsampling.png)

##### Negative Sampling
- each training sample will tweak all of the weights in the neural network
- the size of our word vocabulary means that our skip-gram neural network has a tremendous number of weights, all of which would be updated slightly by every one of our billions of training samples!
- Negative sampling addresses this by having each training sample only modify a small percentage of the weights, rather than all of them
- With negative sampling, we are instead going to randomly select just a small number of “negative” words (let’s say 5) to update the weights for. (In this context, a “negative” word is one for which we want the network to output a 0 for). We will also still update the weights for our “positive” word
- The paper says that selecting 5-20 words works well for smaller datasets, and you can get away with only 2-5 words for large datasets.
- Recall that the output layer of our model has a weight matrix that’s 300 x 10,000. So we will just be updating the weights for our positive word (“quick”), plus the weights for 5 other words that we want to output 0. That’s a total of 6 output neurons, and 1,800 weight values total. That’s only 0.06% of the 3M weights in the output layer!
- In the hidden layer, only the weights for the input word are updated (this is true whether you’re using Negative Sampling or not)